In [ ]:
#Requires to have geopandas installed
#Installation https://geopandas.org/getting_started/install.html

import matplotlib.pyplot as plt 
import geopandas as gpd
import geoplot
import pandas as pd
import fiona #;help(fiona.open)
import requests
from pandas import json_normalize
import requests # convert json into dataframe
#For OSM 
import pyrosm
from pyrosm import OSM, get_data

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

#  Adding MENA to Africa

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world

In [ ]:
saudi = world.query('name == "Saudi Arabia"')
yemen = world.query('name == "Yemen"')
iraq = world.query('name == "Iraq"')
oman = world.query('name == "Oman"')
uae = world.query('name == "United Arab Emirates"')
jordan = world.query('name == "Jordan"')
palestine = world.query('name == "Palestine"')
israel = world.query('name == "Israel"')
qatar = world.query('name == "Qatar"')
kuwait = world.query('name == "Kuwait"')

mena = saudi.append(yemen).append(iraq).append(oman).append(uae).append(jordan).append(palestine).append(israel).append(qatar).append(kuwait)

In [ ]:
africa = world.query('continent == "Africa"').append(mena)
#africa.head()

In [ ]:
### Example plot https://geopandas.org/gallery/plotting_with_geoplot.html?highlight=africa

# #gdp_per_cap = world.gdp_md_est / world.pop_est
# ax = geoplot.cartogram(
#     africa, scale='pop_est', limits=(0.2, 1),
#     edgecolor='None', figsize=(7, 8)
# )
# geoplot.polyplot(africa, edgecolor='gray', ax=ax)

In [ ]:
#ax = africa.plot(color="white", edgecolors="red",figsize=(10, 10))

# 3) Substations - Onsset - OSM

Install PYROSM to handle OSM data: https://pyrosm.readthedocs.io/en/latest/installation.html

data source http://download.geofabrik.de/  

data description: https://gep-onsset.readthedocs.io/en/latest/GIS%20data%20collection.html

### OSM content check

In [ ]:
from pyrosm import OSM, get_data

# Initialize the OSM reader with test data
fp = get_data("Helsinki")
osm = OSM(fp)

# The instance has a configuration attribute containing:
print([item for item in osm.conf.__dict__.keys() if not item.startswith("_")])


In [ ]:
# Show all available tag attributes
#osm.conf.tags.available


In [ ]:
# Show all tags that are converted into columns from building features
osm.conf.tags.power

## Create OSM substation dataframe

Pyrosm has an existing issue handling geofabric data. Thats why "get_data("africa")" is not working. 

A way around this problem is to download the data from http://download.geofabrik.de/africa-latest.osm.bz2 locally & set the path right

://github.com/HTenkanen/pyrosm/issues/109

In [ ]:
pwd

In [ ]:
cd

In [ ]:
cd "OneDrive/PHD-Flexibility/07_pypsa-africa/Data"

In [ ]:
africa_data = "africa-latest.osm.pbf"

In [ ]:
#Possible input to extract data from PYROSM. I.e. get_data("angola") gets the "angola-latest.osm.pbf" filled with all OSM data
regions =     ["algeria",
               "angola",
               "benin",
               "botswana",
               "burkina_faso",
               "burundi",
               "cameroon",
               "canary_islands",
               "cape_verde",
               "central_african_republic",
               "chad",
               "comores",
               "congo_brazzaville",
               "congo_democratic_republic",
               "djibouti",
               "egypt",
               "equatorial_guinea",
               "eritrea",
               "ethiopia",
               "gabon",
               "ghana",
               "guinea_bissau",
               "guinea",
               "ivory_coast",
               "kenya",
               "lesotho",
               "liberia",
               "libya",
               "madagascar",
               "malawi",
               "mali",
               "mauritania",
               "mauritius",
               "morocco",
               "mozambique",
               "namibia",
               "niger",
               "nigeria",
               "rwanda",
               "saint_helena_ascension_and_tristan_da_cunha",
               "sao_tome_and_principe",
               "senegal_and_gambia",
               "seychelles",
               "sierra_leone",
               "somalia",
               "south_africa_and_lesotho",
               "south_africa",
               "south_sudan",
               "sudan",
               "swaziland",
               "tanzania",
               "togo",
               "tunisia",
               "uganda",
               "zambia",
               "zimbabwe"]

In [ ]:
fp = get_data("lesotho")

In [ ]:
fp = '/home/matin/pypsa-africa/Africa-node.osm.pbf'

In [ ]:
osm = pyrosm.OSM(fp)

In [ ]:
my_filter = {'power': ["substation"]}

In [ ]:
lesotho = osm.get_pois(custom_filter=my_filter)

In [ ]:
lesotho.head(5)

In [ ]:
#Shows geometry reference
lesotho['geometry'].crs

In [ ]:
#Converting Polygons to "centroids" = Points. 

#We used "epsg=3035" to make the long/lat numbers larger.
#This was necessary to make the centroid operations more accurate. Only required for small polygons.
#Afterward we reconvert it to the standard epsg=4326

lesotho['geometry'] = lesotho['geometry'].to_crs(epsg=3035)
lesotho['geometry'] = lesotho['geometry'].centroid
lesotho['geometry'] = lesotho['geometry'].to_crs(epsg=4326)

In [ ]:
# Plot
ax = lesotho.plot(column='power', marker='o', markersize=10, figsize=(6,6), legend=True, legend_kwds=dict(loc='upper left', ncol=5, bbox_to_anchor=(1, 1)))

In [ ]:
#Worked! Lesotho is downn i
ax = lesotho.plot(column='power',figsize=(10, 10),alpha = 0.7, markersize = 1)
geoplot.polyplot(africa, edgecolor='gray', ax=ax)

In [ ]:
url = 'https://lz4.overpass-api.de/api/interpreter'  # Overpass API URL


In [ ]:
# Bounding Boxes
# Nigeria (approx) (0.61,-4.21,21.75,15.79)
query = f"""
[out:json];
// gather results
(
  // query part for: “power=substation”
  node["power"="substation"]["voltage"](0.61,-4.21,21.75,15.79);
  way["power"="substation"]["voltage"](0.61,-4.21,21.75,15.79);
  relation["power"="substation"]["voltage"](0.61,-4.21,21.75,15.79);
);
// print results
out body;
"""

In [ ]:
# Bounding Boxes
# Nigeria (approx) (0.61,-4.21,21.75,15.79)
query = f"""
[out:json];
// gather results
(
  // query part for: “power=substation”
  ( area["ISO3166-1"="IT"];) ->.a;
  node["power"="substation"](area.a);
  way["power"="substation"](area.a);
  relation["power"="substation"](area.a);
);
// print results
out body;
"""

In [ ]:
# Africa (approx) (-23.73,-36.03,58.00,35.74)
query = f"""
[out:json];
// gather results
(
  // query part for: “power=substation”
  node["power"="substation"]["voltage"](-23.73,-36.03,58.00,35.74);
  way["power"="substation"]["voltage"](-23.73,-36.03,58.00,35.74);
  relation["power"="substation"]["voltage"](-23.73,-36.03,58.00,35.74);
);
// print results
out body;
"""


In [ ]:
r = requests.get(url, params={'data': query})
data = r.json()['elements'] # read response as JSON and get the data
df = pd.json_normalize(data)  # create a DataFrame from the data

In [ ]:
c_list = ["DZ","AO","SH","BJ","BW","BF","BI","CM","CV","CF","TD","KM","CG","CD","DJ","EG","GQ","ER","SZ","ET","GA","GM","GH","GN","GW","CI","KE","LS","LR","LY","MG","MW","ML","MR","MU","YT","MA","MZ","NA","NE","NG","ST","RE","RW","ST","SN","SC","SL","SO","ZA","SS","SH","SD","SZ","TZ","TG","TN","UG","CD","ZM","TZ","ZW"]
len(c_list)

In [ ]:
print(df.shape)

In [ ]:
# Pretty Print dataframe (vscode)
display(df)

In [ ]:
df1=df.dropna(subset=['tags.voltage'])

In [ ]:
display(df1)

In [ ]:
df.isna().sum()

In [ ]:
# type, id, lat, lon, tags.voltage, node
["voltage"]
df.isna().sum()
df1=df.dropna(subset=['tags.voltage'])